# Simple Benchmarker

This is running on Google's free cloud compute. This notebook must be kept open for the code to keep running.

## Instructions

1. **To run this notebook, you must have a google account and be using Chrome browser**.

  a. Notebooks can be ran continuously for 12 hours on Google Colabs before a reset is required

2. Run the code below (press Ctrl + F9, or press the play button next to "Show code" below)

    a. First time running this notebook, a popup will appear warning about non-Google authored code. Please be assured that none of our code will ever request access to your Google account.



In [ ]:
#@title script to start Simple Benchmarker
import sys
import os
import subprocess

def cloneOrPullTIGRepo(repo_name):
    repo_path = f"{os.getcwd()}/{repo_name}"
    if not os.path.exists(repo_path):
        subprocess.run(["git", "clone", "-b", "master", f"https://github.com/the-innovation-game/{repo_name}.git"], check=True)
    else:
        subprocess.run(["git", "pull", "origin", "master"], cwd=repo_path, check=True)
    if repo_path not in sys.path:
        sys.path.append(repo_path)

cloneOrPullTIGRepo("simple_benchmarker")
cloneOrPullTIGRepo("challenges")

from datetime import datetime, timedelta
from benchmarker_utils.api import API
from benchmarker_utils.misc import logger, calcSeed, randomInterpolate, IntermediateIntegersLogger
from threading import Thread
from uuid import uuid4
import random
import numpy as np
import time

if 'G' not in locals():
  G = {}

# Start Benchmarker
if "BENCHMARKER" not in G:
  class SimpleBenchmarker:
      def __init__(self, api: API):
          self.api = api
          self.running = True
          self.status = "Initialising"
          self.earnings = None
          self.active_algorithms = {}
          self.recent_benchmarks = None
          self.underperformed_benchmarks = {
            'header_row': [
                'id',
                'datetime_submitted',
                'latest_earnings',
                'block_id',
                'challenge_id',
                'algorithm_id',
                'difficulty',
                'num_solutions',
                'frontier_idx',
                'status',
            ],
            'data_rows': []
          }
          self.latest_block = None
          self.frontiers = None
          self.reset()

      def reset(self):
          self.proofs = []
          self.nonce = 0
          self.num_errors = 0
          self.benchmark_start = None
          self.benchmark_end = None
          self.challenge_id = None
          self.algorithm_id = None
          self.difficulty = None

      def pickChallengeToBenchmark(self):
          # picks challenge which we have benchmarked the least
          # this helps us increase our balance score
          min_earnings = min(
              c[1]
              for c in self.earnings["benchmarker_earnings"]["latest_by_challenge"]
          )
          return random.choice([
              c[0]
              for c in self.earnings["benchmarker_earnings"]["latest_by_challenge"]
              if c[1] == min_earnings
          ])

      def pickAlgorithmToBenchmark(self):
          algorithm_ids = [a[0] for a in self.active_algorithms[self.challenge_id]['data_rows']]
          return random.choice(algorithm_ids)

      def pickDifficultyToBenchmark(self):
          benchmarks = [
              {
                  k: d[i]
                  for i, k in enumerate(self.frontiers["header_row"])
              }
              for d in self.frontiers["data_rows"]
          ]

          # group benchmarks by frontier_idx
          benchmarks_by_frontier_idx = {}
          num_solutions_on_frontiers = 0
          for b in benchmarks:
              benchmarks_by_frontier_idx.setdefault(b["frontier_idx"], []).append(b)
              num_solutions_on_frontiers += b["num_solutions"] * (b["frontier_idx"] is not None)
          # LOGGER.debug(f"Got {len(benchmarks)} benchmarks with {num_solutions_on_frontiers} solutions on frontiers")

          # FIXME this assumes Difficulty has exactly 2 parameters
          difficulty_bounds = self.latest_block["difficulty_bounds"][self.challenge_id]
          x_param, y_param = list(difficulty_bounds)
          min_difficulty = {x_param: difficulty_bounds[x_param][0], y_param: difficulty_bounds[y_param][0]}
          max_difficulty = {x_param: difficulty_bounds[x_param][1], y_param: difficulty_bounds[y_param][1]}

          # benchmarks with frontier_idx None do not earn tokens
          # 0 is the easiest difficulty frontier
          if 0 not in benchmarks_by_frontier_idx:
              difficulty = min_difficulty
          else:
              # randomly interpolate a point on the easiest frontier
              random_x, random_y = randomInterpolate(
                  points=[(b["difficulty"][x_param], b["difficulty"][y_param]) for b in benchmarks_by_frontier_idx[1]],
                  min_point=(min_difficulty[x_param], min_difficulty[y_param])
              )
              # randomly increment/decrement difficulty
              pos_or_neg = (-1) ** (num_solutions_on_frontiers < self.latest_block["target_num_solutions"]) # hack to set True=-1, False=1
              difficulty = {
                  x_param: random_x + random.randint(0, 1) * pos_or_neg,
                  y_param: random_y + random.randint(0, 1) * pos_or_neg
              }

              for param in [x_param, y_param]:
                # Ensure our difficulty is within the bounds
                difficulty[param] = int(np.clip(
                    difficulty[param],
                    a_min=min_difficulty[param],
                    a_max=max_difficulty[param]
                ))
          return difficulty

      def generateAndSolveInstance(self, benchmark_params):
          Challenge = __import__(f"{self.challenge_id}.challenge").challenge.Challenge
          solveChallenge = getattr(
              __import__(f"{self.challenge_id}.algorithms.{self.algorithm_id}").algorithms,
              self.algorithm_id
          ).solveChallenge

          seed = calcSeed(**benchmark_params, nonce=self.nonce)
          try:
              logger = IntermediateIntegersLogger()
              c = Challenge.generateInstance(seed, self.difficulty)
              solution = solveChallenge(c, logger.log)
              if c.verifySolution(solution):
                  self.proofs.append(dict(
                      nonce=self.nonce,
                      solution=solution if isinstance(solution, list) else solution.tolist(),
                      intermediate_integers=logger.dump()
                  ))
          except:
              self.num_errors += 1
          self.nonce += 1

      def run_once(self):
          self.status = "Querying latest block"
          self.latest_block = self.api.getLatestBlock()
          time.sleep(0.5)

          self.status = "Querying player earnings"
          self.earnings = self.api.getEarnings()
          time.sleep(0.5)

          self.status = "Querying player benchmarks"
          self.recent_benchmarks = self.api.getRecentBenchmarks()
          time.sleep(0.5)

          self.status = "Picking challenge to benchmark"
          self.challenge_id = self.pickChallengeToBenchmark()
          time.sleep(0.5)

          self.status = "Querying active algorithms"
          self.active_algorithms[self.challenge_id] = self.api.getAlgorithms(self.challenge_id)
          cloneOrPullTIGRepo("challenges")
          time.sleep(0.5)

          self.status = "Picking algorithm to benchmark"
          self.algorithm_id = self.pickAlgorithmToBenchmark()
          time.sleep(0.5)

          self.status = "Querying difficulty frontiers"
          self.frontiers = self.api.getFrontiers(self.challenge_id)
          time.sleep(0.5)

          self.status = "Picking difficulty to benchmark"
          self.difficulty = self.pickDifficultyToBenchmark()
          time.sleep(0.5)

          self.benchmark_start = datetime.now().astimezone()
          self.benchmark_end = self.benchmark_start + timedelta(seconds=60)

          benchmark_params = dict(
              player_id=self.earnings["player_id"],
              block_id=self.latest_block["block_id"],
              prev_block_id=self.latest_block["prev_block_id"],
              challenge_id=self.challenge_id,
              algorithm_id=self.algorithm_id,
              difficulty=self.difficulty
          )

          self.status = "Benchmarking"
          while datetime.now().astimezone() < self.benchmark_end and self.running:
              self.generateAndSolveInstance(benchmark_params)

          if len(self.proofs) > 0:
              self.status = "Submitting Benchmark"
              resp = self.api.submitBenchmark(**benchmark_params, nonces=[p['nonce'] for p in self.proofs])
              sampled_nonces = set(resp["sampled_nonces"])
              self.api.submitProofs(
                  resp["benchmark_id"],
                  proofs=[
                      p for p in self.proofs
                      if p['nonce'] in sampled_nonces
                  ]
              )
          else:
              self.status = "Benchmark underperformed. Skipping submission"
              self.underperformed_benchmarks['data_rows'].append(
                  (
                    str(uuid4()),
                    self.benchmark_end.strftime("%Y-%m-%dT%H:%M:%S.%f%z"),
                    0,
                    self.latest_block['block_id'],
                    self.challenge_id,
                    self.algorithm_id,
                    self.difficulty,
                    0,
                    None,
                    'UNDER-PERFORMED',
                  )
              )
              time.sleep(2)

          while (
              len(self.underperformed_benchmarks['data_rows']) > 0 and
               (datetime.now().astimezone() - datetime.strptime(self.underperformed_benchmarks['data_rows'][0][0], "%Y-%m-%dT%H:%M:%S.%f%z")).total_seconds() > self.latest_block['seconds_benchmark_active']
          ):
              self.underperformed_benchmarks['data_rows'].pop(0)

      def run_forever(self):
          while self.running:
              try:
                  self.reset()
                  self.run_once()
              except Exception as e:
                  self.status = f"Error: {e}"
                  import traceback
                  print(traceback.format_exc())
                  print(e)
                  time.sleep(5)

  G["BENCHMARKER"] = None


# Start Flask Server if it doesn't exist
if "SERVER" not in G:
  from flask import Flask, request
  from werkzeug.serving import make_server

  app = Flask("SimpleBenchmarker")

  @app.route('/get_status', methods=['GET'])
  def getStatus():
    if G['BENCHMARKER'] is None:
        return dict(status="Waiting for API Key")
    else:
      b = G['BENCHMARKER']
      return dict(
            api_key=b.api.api_key,
            status=b.status,
            earnings=b.earnings,
            recent_benchmarks=b.recent_benchmarks,
            underperformed_benchmarks=b.underperformed_benchmarks,
            benchmark_start=None if b.benchmark_start is None else b.benchmark_start.strftime("%Y-%m-%dT%H:%M:%S.%f%z"),
            benchmark_end=None if b.benchmark_end is None else b.benchmark_end.strftime("%Y-%m-%dT%H:%M:%S.%f%z"),
            challenge_id=b.challenge_id,
            algorithm_id=b.algorithm_id,
            difficulty=b.difficulty,
      )

  @app.route('/start_benchmarking/<api_key>', methods=['GET'])
  def startBenchmark(api_key):
    if G['BENCHMARKER'] is None or G['BENCHMARKER'].api.api__key != api_key:
      G['BENCHMARKER'] = SimpleBenchmarker(
          API(api_key, "https://api.the-innovation-game.com/v1")
      )
      Thread(target=G['BENCHMARKER'].run_forever).start()
    return 'OK'

  @app.route('/stop_benchmarking', methods=['GET'])
  def stopBenchmarking():
    if G['BENCHMARKER'] is not None:
      G['BENCHMARKER'].running = False
      G['BENCHMARKER'] = None
    return 'OK'

  G["SERVER"] = make_server("0.0.0.0", 3000, app)
  Thread(target=G["SERVER"].serve_forever).start()

try:
    from IPython.display import display, HTML, clear_output
    import requests
    clear_output(wait=True)
    resp = requests.get("https://the-innovation-game.com/simple-benchmarker")
    resp.encoding = 'UTF-8'
    display(HTML(resp.text))
except ImportError as e:
   logger.error("IPython module does not exist. This script is intended to be ran in a notebook")